In [ ]:
!pip install tensorflow-gpu==1.15.3

     |████████████████████████████████| 411.0MB 43kB/s 
     |████████████████████████████████| 3.8MB 44.6MB/s 
     |████████████████████████████████| 512kB 52.7MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=4f77183c26d45864469072876cb7997af7c3d564abebfcb08b153eb5eecf2359
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [ ]:
!pip install Keras==2.1.5

     |████████████████████████████████| 337kB 8.4MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!git clone https://github.com/worldstar/keras-yolo3.git

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 604.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
%cd keras-yolo3

/content/keras-yolo3


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights yolov3.weights
!python convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5

In [ ]:
!git clone https://github.com/worldstar/dataset.git

Cloning into 'dataset'...
remote: Enumerating objects: 713, done.
remote: Total 713 (delta 0), reused 0 (delta 0), pack-reused 713
Receiving objects: 100% (713/713), 7.26 MiB | 12.63 MiB/s, done.
Resolving deltas: 100% (341/341), done.


In [ ]:
fw = open('model_data/voc_classes.txt', "w")
fw.write('RBC')
fw.close()

In [ ]:
import xml.etree.ElementTree as ET
import os
from os import getcwd
import sys

def generateAnnotations(path, imagePath, writePath, voc_classes):
    # path = sys.argv[1] #"./Data/Annotations/"
    # imagePath = sys.argv[2]#"./Data/JPEGImages/"
    # writePath = sys.argv[3]#"./model_data/train.txt"
    deputyFileName = "jpg"
    # classes = ["bicycle","car","cat","dog","person"]
    fr = open(voc_classes, 'r')#"model_data/voc_classes.txt"
    classes = fr.read().split("\n")
    fr.close()

    fw = open(writePath, "w")
    for fileName in os.listdir(path):
        if fileName in ".gitignore":
            continue
        print("readFile:",(path+fileName))
        convertResult = convert_annotation((path+fileName),classes,imagePath,deputyFileName)
        fw.write(convertResult)
    fw.close()

def dataUs(infos):
    return infos.split(".")[0]

def convert_annotation(path,classes,imagePath,deputyFileName): 
    try: 
        xmlFile = open(path) 
    except:
        xmlFile = open(path,encoding="utf-8")   
    xmlTree = ET.parse(xmlFile)
    xmlRoot = xmlTree.getroot()
    width,height,depth = -1,-1,-1
    hasClass = False
    result = ""
    for xmlObj in xmlRoot.iter('size'):
        width = xmlObj.find('width').text.replace(" ", "").replace("\t", "").replace("\n", "")
        height = xmlObj.find('height').text.replace(" ", "").replace("\t", "").replace("\n", "").replace(" ", "").replace("\t", "").replace("\n", "")
        depth = xmlObj.find('depth').text.replace(" ", "").replace("\t", "").replace("\n", "")
        # print(width,height,depth)
    for xmlObj in xmlRoot.iter('object'):
        name = xmlObj.find('name').text.replace(" ", "").replace("\t", "").replace("\n", "")
        isClass = False
        classNum = -1
        for i in range(0,len(classes),1):
            if(name == classes[i]):
                isClass = True
                hasClass = True
                classNum = i
        if(isClass):
            xmin , ymin , xmax , ymax = -1 , -1 , -1 , -1
            for xmlObj2 in xmlObj.iter('bndbox'):
                xmin = int(dataUs(xmlObj2.find('xmin').text.replace(" ", "").replace("\t", "").replace("\n", "").replace(" ", "").replace("\t", "").replace("\n", "")))
                ymin = int(dataUs(xmlObj2.find('ymin').text.replace(" ", "").replace("\t", "").replace("\n", "")))
                xmax = int(dataUs(xmlObj2.find('xmax').text.replace(" ", "").replace("\t", "").replace("\n", "")))
                ymax = int(dataUs(xmlObj2.find('ymax').text.replace(" ", "").replace("\t", "").replace("\n", "")))
            result += " %s,%s,%s,%s,%d"%(xmin,ymin,xmax,ymax,classNum)
    if(hasClass):
        FileName = os.path.basename(path)
        FileName = os.path.splitext(FileName)[0]
        FileName = FileName+"."+deputyFileName
        result = "%s%s\n"%((imagePath + FileName),result)
    return result

In [ ]:
generateAnnotations('dataset/Annotations/', 'dataset/JPEGImages/', 'train.txt', 'model_data/voc_classes.txt')

In [ ]:
!python train.py #WPlease note there are two stages, the first stage runs 20 epochs and the 2nd stage executes 100 epochs.

Using TensorFlow backend.








2021-02-25 03:44:13.524316: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-02-25 03:44:13.529010: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-02-25 03:44:13.529214: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562fbe8fea00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-02-25 03:44:13.529243: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-02-25 03:44:13.530934: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-02-25 03:44:13.666199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node